In [ ]:
from common import prepare_val
import torch

device = torch.device('cuda:0') 
exp_id = '2025_0331_145017'
CHECKPOINT_PATH = f"../logs/GaussianLSS/{exp_id}/checkpoints/last.ckpt"
overrides = ['visualization.keys=[lane, road_segment, vehicle]', '+data.map_layers=[lane, road_segment]']
model, network, loader, viz, dataset = prepare_val('GaussianLSS', device, CHECKPOINT_PATH, overrides=overrides, mode='split', batch_size=1)

In [ ]:
import time

GIF_PATH = 'predictions.gif'
network.eval()
network.to(device)
images = []
with torch.no_grad():
    for i,batch in enumerate(loader):
        print(i,end='\r')
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(device)
            elif isinstance(v, list):
                if isinstance(v[0], torch.Tensor):
                    batch[k] = [i.to(device) for i in v]
            else:
                batch[k] = v
        pred = network(batch)
        vis_result = viz(batch,pred)
        images += vis_result

In [ ]:
from PIL import Image

frames = [Image.fromarray(f) for f in images]  # or use Image.fromarray(np_array)
frames[0].save(
    GIF_PATH,
    save_all=True,
    append_images=frames[1:],
    duration=750,   # milliseconds
    loop=0          # 0 = infinite loop
)